In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing, utils
from sklearn.metrics import  make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_av = pd.read_csv("avila.txt", header=None)
df_av

,0,1,2,3,4,5,6,7,8,9,10
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
10425,0.080916,0.588093,0.015130,0.002250,0.261718,-0.557133,0.371178,0.932346,0.282354,-0.580141,F
10426,0.253730,-0.338346,0.352988,-1.154243,0.172340,-0.557133,0.257927,0.348428,0.032902,-0.527134,F
10427,0.229043,-0.000745,0.171611,-0.002793,0.261718,0.688613,0.295677,-1.088486,-0.590727,0.580142,A
10428,-0.301743,0.352558,0.288973,1.638181,0.261718,0.688613,0.069175,0.502761,0.625350,0.718969,E


In [3]:

idx=  np.array(df_av.iloc[:,-1])
y = np.zeros((len(idx)), dtype=np.int32)
y[idx=='A'] = 1
y

array([1, 1, 1, ..., 1, 0, 0], dtype=int32)

In [4]:
#divide data into x and y
x = df_av.iloc[:,:-1]

#seeing %pos in the data
pos = y.sum()
neg = len(y) - pos

percent_pos = round(pos/len(y)*100, 2)
percent_neg = round(neg/len(y)*100, 2)

percent_pos

41.09

In [5]:
x_std = preprocessing.scale(x).astype(np.float32)
y_std = y

In [6]:
result_lr = []
for trial in range(5):
    #splitting into training and testing
    x_train, x_test, y_train, y_test = train_test_split(x_std, y_std, train_size = 5000, random_state=trial,
                                                    stratify = y_std, shuffle=True)

    #encode training and testing data != continuous
    encode_ytr = preprocessing.LabelEncoder()
    ytr_encoded = encode_ytr.fit_transform(y_train)
    
    encode_yt = preprocessing.LabelEncoder()
    yt_encoded = encode_yt.fit_transform(y_test)
    
    C_list = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, np.Inf]
    alg = LogisticRegression(max_iter=5000, solver='saga')
    param_grid =  {'penalty' : ['l1', 'l2'], 'C': C_list}
    scoring_list = ['accuracy', 'precision', 'f1', 'roc_auc', 'recall']
    scoring_func_dict = {'acc':accuracy_score, 'AUC':roc_auc_score,
                        'prec':precision_score, 'recall':recall_score, 'f1':f1_score}
    scoring_dict = {k:make_scorer(v) for k,v in scoring_func_dict.items()}
    
    search_result = GridSearchCV (alg, param_grid, scoring=scoring_dict, refit=False, cv=5, n_jobs=-1)
    search_result.fit(x_train, ytr_encoded)
    
    res_one_trial = []
    for k in scoring_dict.keys():
        
        mean_test_metric = search_result.cv_results_['mean_test_'+k]
        best_idx = np.argmax(mean_test_metric)
        best_params = search_result.cv_results_['params'][best_idx]
        alg_new_5000 = LogisticRegression(max_iter=5000, solver='saga', **best_params)
        alg_new_5000.fit(x_train, ytr_encoded)
        
        res_test = alg_new_5000.predict(x_test)
        
        res_one_model = []
        
        try:
            for k_metric, func_metric in scoring_func_dict.items():
                metric_value = func_metric(res_test, yt_encoded)
                res_one_model.append(metric_value)
        except ValueError:
            pass

        res_one_trial.append(res_one_model)
    
    result_lr.append(res_one_trial)
result_lr = np.array(result_lr)
print(result_lr.shape)
print(np.mean(result_lr, axis=0))

(5, 5, 5)
[[0.68946593 0.68070345 0.51340206 0.65608241 0.57600419]
 [0.69005525 0.68108994 0.517974   0.65545227 0.57862526]
 [0.68445672 0.67677426 0.48839086 0.65604819 0.55896551]
 [0.69001842 0.68105095 0.51788436 0.65541231 0.57855316]
 [0.69005525 0.68109616 0.51788436 0.65548775 0.57858387]]


In [7]:
print([k for  k in scoring_dict.keys()])
print([k for k in scoring_func_dict.keys()])

['acc', 'AUC', 'prec', 'recall', 'f1']
['acc', 'AUC', 'prec', 'recall', 'f1']
